In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

In [2]:
IMAGE_SIZE = 28
IMAGE_CHANNEL_NUM = 1

INPUT_SIZE = IMAGE_SIZE * IMAGE_SIZE
OUTPUT_SIZE = 10

CONV_1_SIZE = 5
CONV_1_DEPTH = 32

CONV_2_SIZE = 5
CONV_2_DEPTH = 64

FC_SIZE = 512

In [3]:
def get_weight(shape):
    return tf.get_variable("weight", shape, initializer=tf.truncated_normal_initializer(stddev=0.1))

In [4]:
def get_bias(shape):
    return tf.get_variable("bias", shape, initializer=tf.constant_initializer(0.0))

In [5]:
def conv2d(input_tensor, weight):
    return tf.nn.conv2d(input_tensor, weight, strides=[1, 1, 1, 1], padding="SAME")

In [6]:
def max_pool_2x2(input_tensor):
    return tf.nn.max_pool(input_tensor, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

In [7]:
def inference(input_tensor, train, regularizer):
    with tf.variable_scope("layer_1_conv"):
        conv_1_weight = get_weight([CONV_1_SIZE, CONV_1_SIZE, IMAGE_CHANNEL_NUM, CONV_1_DEPTH])
        conv_1_bias = get_bias([CONV_1_DEPTH])
        conv_1 = conv2d(input_tensor, conv_1_weight)
        conv_1_activation = tf.nn.relu(tf.nn.bias_add(conv_1, conv_1_bias))
    with tf.variable_scope("layer_1_pool"):
        pool_1 = max_pool_2x2(conv_1_activation)
    with tf.variable_scope("layer_2_conv"):
        conv_2_weight = get_weight([CONV_2_SIZE, CONV_2_SIZE, CONV_1_DEPTH, CONV_2_DEPTH])
        conv_2_bias = get_bias([CONV_2_DEPTH])
        conv_2 = conv2d(pool_1, conv_2_weight)
        conv_2_activation = tf.nn.relu(tf.nn.bias_add(conv_2, conv_2_bias))
    with tf.variable_scope("layer_2_pool"):
        pool_2 = max_pool_2x2(conv_2_activation)
        pool_2_shape = pool_2.get_shape().as_list()
        nodes = pool_2_shape[1] * pool_2_shape[2] * pool_2_shape[3]
        pool_2_reshaped = tf.reshape(pool_2, [-1, nodes])
    with tf.variable_scope("layer_3_fc"):
        fc_1_weight = get_weight([nodes, FC_SIZE])
        if regularizer != None:
            tf.add_to_collection("losses", regularizer(fc_1_weight))
        fc_1_bias = get_bias([FC_SIZE])
        fc_1 = tf.matmul(pool_2_reshaped, fc_1_weight) + fc_1_bias
        fc_1_activation = tf.nn.relu(fc_1)
        if train:
            fc_1_activation = tf.nn.dropout(fc_1_activation, 0.5)
    with tf.variable_scope("layer_4_fc"):
        fc_2_weight = get_weight([FC_SIZE, OUTPUT_SIZE])
        if regularizer != None:
            tf.add_to_collection("losses", regularizer(fc_2_weight))
        fc_2_bias = get_bias([OUTPUT_SIZE])
        logits = tf.matmul(fc_1_activation, fc_2_weight) + fc_2_bias
    return logits

In [8]:
BATCH_SIZE = 100
LEARNING_RATE_BASE = 0.01
LEARNING_RATE_DECAY = 0.99
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 3000
MOVING_AVERAGE_DECAY = 0.99

In [9]:
def train(mnist):
    x = tf.placeholder(tf.float32, [None, IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNEL_NUM], name="x")
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_SIZE], name="y_")
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    y = inference(x, False, regularizer)
    global_step = tf.Variable(0, trainable=False)
    variable_ave = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    variable_ave_op = variable_ave.apply(tf.trainable_variables())
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=y, labels=tf.argmax(y_, 1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    loss = cross_entropy_mean + tf.add_n(tf.get_collection("losses"))
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE, global_step, mnist.train.num_examples / BATCH_SIZE, 
        LEARNING_RATE_DECAY, staircase=True)
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(
        loss, global_step=global_step)
    with tf.control_dependencies([train_step, variable_ave_op]):
        train_op = tf.no_op(name='train')

    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1), name='correct_prediction')
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        for i in range(1, TRAINING_STEPS + 1):
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            xs_reshaped = np.reshape(xs, [BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNEL_NUM])
            _, loss_value, acc_value, step = sess.run([train_op, loss, accuracy, global_step], 
                                                      feed_dict={x: xs_reshaped, y_: ys})
            if i % 200 == 0:
                    print("Setp %d, loss = %g, accuracy = %g." % (step, loss_value, acc_value))
        x_test = np.reshape(mnist.test.images, [len(mnist.test.images), IMAGE_SIZE, 
                                                IMAGE_SIZE, IMAGE_CHANNEL_NUM])
        y_test = mnist.test.labels
        test_acc_value = sess.run(accuracy, feed_dict={x: x_test, y_: y_test})
        print("Test accuracy = {}".format(test_acc_value))

In [10]:
mnist = input_data.read_data_sets("./data", one_hot=True)
train(mnist)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Setp 200, loss = 0.999788, accuracy = 0.89.
Setp 400, loss = 0.849933, accuracy = 0.95.
Setp 600, loss = 0.702903, accuracy = 0.97.
Setp 800, loss = 0.762394, accuracy = 0.97.
Setp 1000, loss = 0.704916, accuracy = 0.98.
Setp 1200, loss = 0.745765, accuracy = 0.97.
Setp 1400, loss = 0.709202, accuracy = 0.97.
Setp 1